## Yashika Khurana
### Plagiarism Detection

### setting up the environment

In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')
#conf.set('spark.sql.repl.eagerEval.enabled', True) run these two lines next week
#conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf) #spark context
spark = pyspark.sql.SparkSession(sc) #to start sql session
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/30 09:13:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.6/769.6 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 10.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### importing required libraries

In [2]:
from datasketch import MinHash, MinHashLSH
import pyspark
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, DoubleType, IntegerType
import re
import nltk

### Using stopwords to ignore insignificant words

In [3]:
stopwords=nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from nltk.corpus import stopwords

In [6]:
stop_words = set(stopwords.words('english'))

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()

23/03/30 09:15:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [22]:
df = spark.read.option("multiline", True)\
.option('escape', '"')\
.option("header", True)\
.option("InferSchema", True)\
.csv("shared/midterm/articles1.csv")\
.select("id", "title", "publication", "author", "date", "content")\
.withColumn("date", to_date(col("date"), "YYYY-MM-DD"))

### Filtering out the reference article

In [23]:
ref_article = df.filter(df.id == 69716).select('content').head()[0]

### Function to preprocess the content

In [24]:
def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z]', ' ', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [w for w in words if not w in stop_words]
    # Join words back into a string
    text = ' '.join(words)
    return text

In [25]:
preprocess_udf = udf(preprocess)

### Calculating the jaccard similarity

In [26]:
def compute_jaccard(ref, content):
    # Create MinHash objects for the reference article and the current article
    m1 = MinHash()
    for word in ref.split():
        m1.update(word.encode('utf8'))
    m2 = MinHash()
    for word in content.split():
        m2.update(word.encode('utf8'))
    # Compute Jaccard similarity
    return m1.jaccard(m2)

In [27]:
ref_article = preprocess(ref_article)

In [28]:
df = df.withColumn('content_processed', preprocess_udf(df['content']))

### Removing reference article fom the dataframe

In [29]:
df = df.filter(df.id != 69716)

In [30]:
df = df.withColumn('ref_article', udf(lambda x: ref_article, StringType())(col('id')))

In [31]:
df.head(n=1)

[Row(id=17283, title='House Republicans Fret About Winning Their Health Care Suit - The New York Times', publication='New York Times', author='Carl Hulse', date=datetime.date(2016, 12, 31), content='WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stav

### Applying udf to the dataframe

In [35]:
compute_jaccard_udf = udf(compute_jaccard)

In [36]:
df = df.withColumn('jaccard_similarity', compute_jaccard_udf(df['ref_article'], df['content_processed']))

In [37]:
df.head(n=1)

23/03/30 09:33:15 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 9 (TID 9): Attempting to kill Python Worker


[Row(id=17283, title='House Republicans Fret About Winning Their Health Care Suit - The New York Times', publication='New York Times', author='Carl Hulse', date=datetime.date(2016, 12, 31), content='WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stav

### sorting for top 10 similar articles

In [38]:
df = df.orderBy('jaccard_similarity', ascending=False)


In [40]:
top_10_articles = df.select('id', 'title','jaccard_similarity').limit(10)

### results

In [42]:
for article in top_10_articles.collect():
    print(f"Article ID: {article['id']}, Title: {article['title']}, Jaccard Similarity: {article['jaccard_similarity']}")

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:458)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:551)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:519)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


Article ID: 72796, Title: Move over, Vine — people are raking in thousands of dollars a week on a new video app, Jaccard Similarity: 0.1875
Article ID: 52787, Title: CNN 10 - March 27, 2017, Jaccard Similarity: 0.1640625
Article ID: 20153, Title: The World Is Full of Dogs Without Collars - The New York Times, Jaccard Similarity: 0.15625
Article ID: 22807, Title: Abortion Is Found to Have Little Effect on Women’s Mental Health - The New York Times, Jaccard Similarity: 0.15625
Article ID: 63815, Title: Majlinda Kelmendi fights for Kosovo recognition, Jaccard Similarity: 0.15625
Article ID: 17740, Title: Vivian Howard, a TV Chef, Offers Hope for Her Rural Hometown - The New York Times, Jaccard Similarity: 0.1484375
Article ID: 32543, Title: Trump’s Military Rebuild in Limbo in First Hundred Days, Jaccard Similarity: 0.1484375
Article ID: 19176, Title: When Is a Drought Over? A Wet California Wants to Know - The New York Times, Jaccard Similarity: 0.1484375
Article ID: 58415, Title: Donald